# Alineamiento de secuencias patológicas

Comenzamos realizando la carga de paquetes necesarios:

In [1]:
import Bio
from Bio import SeqIO 
from Bio.Seq import Seq

In [2]:
pip show Biopython

Name: biopython
Version: 1.83
Summary: Freely available tools for computational molecular biology.
Home-page: https://biopython.org/
Author: The Biopython Contributors
Author-email: biopython@biopython.org
License: 
Location: c:\users\gabriel\anaconda3\lib\site-packages
Requires: numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


Para obtener las secuencias utilizaremos la siguiente función:

In [3]:
help(SeqIO.parse)

Help on function parse in module Bio.SeqIO:

parse(handle, format, alphabet=None)
    Turn a sequence file into an iterator returning SeqRecords.
    
    Arguments:
     - handle   - handle to the file, or the filename as a string
       (note older versions of Biopython only took a handle).
     - format   - lower case string describing the file format.
     - alphabet - no longer used, should be None.
    
    Typical usage, opening a file to read in, and looping over the record(s):
    
    >>> from Bio import SeqIO
    >>> filename = "Fasta/sweetpea.nu"
    >>> for record in SeqIO.parse(filename, "fasta"):
    ...    print("ID %s" % record.id)
    ...    print("Sequence length %i" % len(record))
    ID gi|3176602|gb|U78617.1|LOU78617
    Sequence length 309
    
    For lazy-loading file formats such as twobit, for which the file contents
    is read on demand only, ensure that the file remains open while extracting
    sequence data.
    
    If you have a string 'data' containin

Comenzamos visualizando nuestras secuecnias patológicas, para los genes correspondientes:

In [4]:
#para nuestro caso únicamente serán dos tipos de extensión/formato; fasta o genbank
def obtener_tipo_extension(extension):
    diccionario_extensiones = {'fasta':'fasta','fna':'fasta','ffn':'fasta','faa':'fasta','frn':'fasta',
                  'fastq':'fastq','fq':'fastq',
                  'gb':'genbank','gbk':'genbank'}
    return diccionario_extensiones[extension]

def obtener_secuencias(ruta_archivo):
    nombre = ruta_archivo.split('.')
    extension = nombre[-1]
    tipo_ext = obtener_tipo_extension(extension)
    registros=SeqIO.parse(ruta_archivo,tipo_ext)
    lista_secuencias=[]
    for secuencia in registros:
        lista_secuencias.append(secuencia)
    return lista_secuencias

Con ello llamamos a la función para el archivo fasta con las secuencias patológicas de los diferentes genes, y los guardamos en una lista, para facilitar su manipulación:

In [5]:
#llamamos a la función
secs_brca1=obtener_secuencias("Datos\Datos pacientes enfermos\BRCA1 sequences.fasta")

#calculamos la cancidad de secuencias presentes aquí
print(f"En el archivo del gen BRCA1 disponemos de un total de {len(secs_brca1)} secuencias patológicas")

#llamamos a la función
secs_brca2=obtener_secuencias("Datos\Datos pacientes enfermos\BRCA2 sequences.fasta")

#calculamos la cancidad de secuencias presentes aquí
print(f"En este archivo del gen BRCA2 disponemos de un total de {len(secs_brca2)} secuencias patológicas")

#llamamos a la función
secs_tp53=obtener_secuencias("Datos\Datos pacientes enfermos\TP53 sequences.fasta")

#calculamos la cancidad de secuencias presentes aquí
print(f"En este archivo del gen TP53 disponemos de un total de {len(secs_tp53)} secuencias patológicas")

#llamamos a la función
secs_pik3ca=obtener_secuencias("Datos\Datos pacientes enfermos\PIK3CA sequences.fasta")

#calculamos la cancidad de secuencias presentes aquí
print(f"En este archivo del gen PIK3CA disponemos de un total de {len(secs_pik3ca)} secuencias patológicas")

En el archivo del gen BRCA1 disponemos de un total de 1099 secuencias patológicas
En este archivo del gen BRCA2 disponemos de un total de 220 secuencias patológicas
En este archivo del gen TP53 disponemos de un total de 953 secuencias patológicas
En este archivo del gen PIK3CA disponemos de un total de 57 secuencias patológicas


# Filtrado de las secuencias de interés

Comenzamos filtrando por aquellas secuencias correspondientes a exones de los genes de interés:

In [6]:
num_exones_brca1=list(filter(lambda secuencia: "exon" in secuencia.description.split(" "),secs_brca1))
print(f'Para el filtrado del gen BRCA1 contamos con un total de {len(num_exones_brca1)} secuencias exónicas')

num_exones_brca2=list(filter(lambda secuencia: "exon" in secuencia.description.split(" "),secs_brca2))
print(f'Para el filtrado del gen BRCA2 contamos con un total de {len(num_exones_brca2)} secuencias exónicas')

num_exones_tp53=list(filter(lambda secuencia: "exon" in secuencia.description.split(" "),secs_tp53))
print(f'Para el filtrado del gen TP53 contamos con un total de {len(num_exones_tp53)} secuencias exónicas')

num_exones_pik3ca=list(filter(lambda secuencia: "exon" in secuencia.description.split(" "),secs_pik3ca))
print(f'Para el filtrado del gen PIK3CA contamos con un total de {len(num_exones_pik3ca)} secuencias exónicas')

Para el filtrado del gen BRCA1 contamos con un total de 394 secuencias exónicas
Para el filtrado del gen BRCA2 contamos con un total de 122 secuencias exónicas
Para el filtrado del gen TP53 contamos con un total de 20 secuencias exónicas
Para el filtrado del gen PIK3CA contamos con un total de 4 secuencias exónicas


In [7]:
def exones(num_exones_gen):
    num_exones=[]
    for exon in num_exones_gen:
        indice_num_exon = exon.description.split(" ").index("exon")+1
        num_exones.append(["exon "+exon.description.split(" ")[indice_num_exon],exon.seq])
    return num_exones

exones_brca1=exones(num_exones_brca1)
exones_brca2=exones(num_exones_brca2)
exones_tp53=exones(num_exones_tp53)
exones_pik3ca=exones(num_exones_pik3ca)

In [8]:
def crear_dic(exones_gen):

    dict_exones={}
    
    for exon in exones_gen:
        
        if exon[0] not in dict_exones.keys():
            dict_exones[exon[0]]=[]
            
        dict_exones[exon[0]].append(exon[1])
    
    return dict_exones
    
dict_brca1=crear_dic(exones_brca1)
dict_brca2=crear_dic(exones_brca2)
dict_tp53=crear_dic(exones_tp53)
dict_pik3ca=crear_dic(exones_pik3ca)

In [9]:
#ejemplo
a=[1,2,4,2,7,2,99,53,12,30,18,2828,2]
a.sort()
print(a[::-1][:2])

[2828, 99]


In [10]:
def max_secs(diccionario,corte):
    num_secs=[]
    max_exones=[]
    
    for exon in diccionario.values():
        num_secs.append(len(exon))
    """
    #ejemplo
    a=[1,2,4,2,7,2,99,53,12,30,18,2828,2]
    a.sort()
    print(a[::-1][:2])
    """
    num_secs.sort()
    num_secs_top= num_secs[::-1][:corte]
    
    secuencias_filtradas=list(filter(lambda secuencias: len(secuencias) in num_secs_top ,diccionario.values()))
    
    for secuencias in secuencias_filtradas:
        max_exones.append(list(diccionario.keys())[list(diccionario.values()).index(secuencias)])
                                  
    return max_exones
              
print(f"Los exones con mayor frecuencia de mutacion para el gen BRCA1 son : {max_secs(dict_brca1,2)}")
print(f"Los exones con mayor frecuencia de mutacion para el gen BRCA2 son : {max_secs(dict_brca2,2)}")
print(f"Los exones con mayor frecuencia de mutacion para el gen TP53 son : {max_secs(dict_tp53,2)}")
print(f"Los exones con mayor frecuencia de mutacion para el gen PIK3CA son : {max_secs(dict_pik3ca,2)}")

Los exones con mayor frecuencia de mutacion para el gen BRCA1 son : ['exon 19', 'exon 5']
Los exones con mayor frecuencia de mutacion para el gen BRCA2 son : ['exon 11', 'exon 2']
Los exones con mayor frecuencia de mutacion para el gen TP53 son : ['exon 8', 'exon 5']
Los exones con mayor frecuencia de mutacion para el gen PIK3CA son : ['exon 9', 'exon 20']


Una vez hemos obtenido los exones de los que disponemos mayores secuencias, y por ello mayor diversidad de estas; las filtramos para poder utilizarlas en nuestros alineamientos de secuencias en busca de posibles patrones génicos.

In [11]:
secuencias_brca1= dict(filter(lambda elemento: elemento[0] == max_secs(dict_brca1,2)[0] or elemento[0] == max_secs(dict_brca1,2)[1],map(lambda elemento: (elemento,dict_brca1[elemento]),dict_brca1)))
secuencias_brca2= dict(filter(lambda elemento: elemento[0] == max_secs(dict_brca2,2)[0] or elemento[0] == max_secs(dict_brca2,2)[1],map(lambda elemento: (elemento,dict_brca2[elemento]),dict_brca2)))
secuencias_tp53= dict(filter(lambda elemento: elemento[0] == max_secs(dict_tp53,2)[0] or elemento[0] == max_secs(dict_tp53,2)[1],map(lambda elemento: (elemento,dict_tp53[elemento]),dict_tp53)))
secuencias_pik3ca= dict(filter(lambda elemento: elemento[0] == max_secs(dict_pik3ca,2)[0] or elemento[0] == max_secs(dict_pik3ca,2)[1],map(lambda elemento: (elemento,dict_pik3ca[elemento]),dict_pik3ca)))

Para ir finalizando la etapa de filtrado y antes de realizar el alineamiento múltiple de nuestras secuencias patológicas filtradas, debemos comprobar que las longitudes de las secuencias para cada exón de interés correspondientes no presentan grandes variaciones entre sí, y de ser así, aquellas que no cumplieran el criterio, deberían de ser descartadas.

In [29]:
#ejemplo comprobar la longitud de las secuencias
for i in secuencias_brca2["exon 2"]:
    print(len(i))

256
205
228
235
228
204
178
212
178
207
192
216
226
226
236
1106


En el ejemplo anterior observamos que alguna secuencia, dista mucho de ser similar en cuanto a longitud se refiere. Por ello debemos por en práctica la idea comentada.

Para la realización de un correcto alineamiento es imprescindible que las longitudes correspondientes a las secuencias sean lo más similares posibles,por ello hacemos mediante la siguiente función, un filtrado de aquellas secuencias que no cumplan con el umbral establecido al presentar longitudes muy diferentes respecto del resto.

In [128]:
import statistics as stats

def calcular_media(dict_secuencias):
    medias=[]
    for j in list(dict_secuencias):
        long=[]
        for i in dict_secuencias[j]:
            long.append(len(str(i)))
        media=stats.mean(long)
        medias.append(media)
    return medias

In [129]:
#función
import statistics as stats

def comprobar_longitudes(dict_secuencias,umbral):
    
    dict={}
    medias=calcular_media(dict_secuencias)
    
    for j in list(dict_secuencias):
        
        long_filtradas=[]
        
        for i in dict_secuencias[j]:
            
            if medias[list(dict_secuencias).index(j)] - umbral <= len(i) <= medias[list(dict_secuencias).index(j)] + umbral:
                
                long_filtradas.append(i)
                
        dict[j]=long_filtradas
        
    return dict
    

In [131]:
#comprobación y filtrado
secuencias_brca1_filtradas=comprobar_longitudes(secuencias_brca1,20)

secuencias_brca2_filtradas=comprobar_longitudes(secuencias_brca2,50)

secuencias_tp53_filtradas=comprobar_longitudes(secuencias_tp53,20)

secuencias_pik3ca_filtradas=comprobar_longitudes(secuencias_pik3ca,20)

#print(secuencias_pik3ca_filtradas)

{'exon 9': [Seq('CCAGAGGGGAAAAATATGACAAAGAAAGCTATATAAGATATTATTTTATTTTAC...ATG'), Seq('CCAGAGGGGAAAAATATGACAAAGAAAGCTATATAAGATATTATTTTATTTTAC...ATG'), Seq('CCAGAGGGGAAAAATATGACAAAGAAAGCTATATAAGATATTATTTTATTTTAC...ATG')], 'exon 20': [Seq('CATTTGCTCCAAACTGACCAAACTGTTCTTATTACTTATAGGTTTCAGGAGATG...ACC')]}


Por otro lado, a continuación,debo volver a crear un archivo fasta, el cual contenga mis secuencias filtradas de interés, para poder hacer uso de la herramienta especificada. Para ello, haremos un filtro del archivo original con las distintas secuencias almacenadas en los diccionarios finales, añadiéndolas a un nuevo archivo fasta creado mediante la función **SeqIO.write()**; y posteriormente le guardaremos.

Con la idea ya en mente, creamos la función:

In [13]:
def almacenar_sec(diccionario,secuencias_originales,gen):
    
    contador=0
    
    lista_keys=list(diccionario.keys())
    
    num_llave=0
    
    while num_llave < len(lista_keys):
            
        lista_sec=[]
          
        for secuencia in secuencias_originales:
        
            
            for sec_filtrada in diccionario[lista_keys[num_llave]]:
                
                #print(str(sec_filtrada))
                
                if str(secuencia.seq)==str(sec_filtrada):
                    
                    contador=contador+1
                    
                    lista_sec.append(secuencia)
                    
            
            SeqIO.write(lista_sec,f"Datos\Secuencias filtradas\{gen.upper()} {lista_keys[num_llave]}.fasta","fasta")
                    
        num_llave=num_llave+1
          

In [134]:
almacenar_sec(secuencias_brca1_filtradas,secs_brca1,"brca1")
almacenar_sec(secuencias_brca2_filtradas,secs_brca2,"brca2")
almacenar_sec(secuencias_tp53_filtradas,secs_tp53,"tp53")
almacenar_sec(secuencias_pik3ca_filtradas,secs_pik3ca,"pik3ca")

De esta forma obteermos finalmente las secuencias completamente filtradas:

In [135]:
secs_brca1

[SeqRecord(seq=Seq('AAGATCTTCTGATCCAGTAGTGTTCTGGACATTGGACTGCTTGTCCCTGGGAAG...TTG'), id='OM524595.1', name='OM524595.1', description='OM524595.1 Homo sapiens isolate Sh520B1 breast cancer type 1 susceptibility protein (BRCA1) gene, exon 20 and partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('AAGATCTTCTGAATCCATGTAGTGTTCTGGACATTGGACTGCTTGTCCCTGGGA...GGC'), id='OM524594.1', name='OM524594.1', description='OM524594.1 Homo sapiens isolate Sh420B1 breast cancer type 1 susceptibility protein (BRCA1) gene, exon 20 and partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('AAGATCTTCTGAATCCATGTAGTGTTCTGGACATTGGACTGCTTGTCCCTGGGA...AGT'), id='OM524593.1', name='OM524593.1', description='OM524593.1 Homo sapiens isolate Sh320B1 breast cancer type 1 susceptibility protein (BRCA1) gene, exon 20 and partial cds', dbxrefs=[]),
 SeqRecord(seq=Seq('GTAGTGTTCTGGACATTGGACTGCTTGTCCCTGGGAAGTAGCAGCAGAAATCAT...AGC'), id='OM524592.1', name='OM524592.1', description='OM524592.1 Homo sapiens isolate Sh220B1 breast cancer typ

# Alineamiento de secuencias

Un alineamiento de secuencias en bioinformática es una forma de representar y comparar dos o más secuencias o cadenas de ADN, ARN, o estructuras primarias proteicas para resaltar sus zonas de similitud, que podrían indicar relaciones funcionales o evolutivas entre los genes o proteínas consultados.

### Clases de alineamiento.

Exiten dos tipos:

- **Alineamiento global**. Se comparan las secuencias enteras, pudiendo introducirse huecos que igualen longitudes de secuencias.
       **(Busca maximizar la puntuación del alinemiento para la secuencia completa)**
    - Coste computacional elevado.
    - Indicado para secuencias con alta similitud y de longitud parecida.

- **Alineamiento local**. Permite identificar secuencias con cierto grado de similitud. Se alinean las partes más parecidas.
      **(Busca maximizar la puntuación del alinemiento para regiones concretas)**
    - Util para busquedas en bases de datos de gran tamaño.
    - En realidad es un alineamiento global de secuencias cortas.


En nuestro caso realizaremos en primer lugar un alineamiento global, esto en pro de obtener las diferencias entre las distintas variantes. Posteriormente pasaremos a un alineamiento local, esta vez para poder determinar dentro de las secuencias aquellas regiones muy similares que puedan ser reconocidas, tras un análisis exahustivo, como biomarcadores patológicos de cáncer de mama.

A su vez, el número de secuencias a confrontar en nuestra situación es múltiple.

Definimos un alineamiento múltiple como una colección de tres o más secuencias de aminoacidos o nucleotidos alineados de forma parcial o completa.

## ¿Para qué puede servir el alineamiento múltiple?

- Informa sobre la función, estructura y evolución de una secuencia.
    - **Identificar patrones comunes de un grupo de secuencias (Para nuestro caso)**
    - Encontrar miembros distantes de una familia de proteinas
        - No es posible obtenerlo mediante el alineamiento por pares.
- Clasificar y generar bases de datos de proteinas una vez secuenciado el genoma completo de un organismo.
    - Predicción de estructuras secundarias y terciarias de proteinas.
- Fundamental para la generación de árboles filogenéticos.
    - Relaciones evolutivas entre secuencias.
- Permite representar familias de secuenccias para generar modelos que faciliten la identificación de miembros potenciales.
    

En el siguiente apartado trataremos con algoritmos de alineamiento múltiple global, es decir que funcionarán mejor cuando las secuencias a comparar tengan un tamaño similar. Asimismo, estos métodos se encuentran diseñados con la premisa de que el numero de *indels* (mutaciones) no sea muy elevado.

**Existen diferentes formas de obtener el alineamiento múltiple**, diferenciando los tipos de algoritmos a emplear:
- Programación dinámica (me sacan la mejor solución, cosa que no siempre se puede hacer en alineamiento múltiple)
- **Algoritmos heurísticos (sacrifican precisión a cambio de rendimiento, nos da una solución que puede que no sea la mejor)**

## Algoritmos heurísticos (no nos aseguran la mejor solución pero nos devuelven una posible solución)

Es lo que se suele utilizar en la realidad.Dado que es posible que nos encontremos con la necesidad de realizar el alineamiento de un gran número de secuencias, no es posible emplear los métodos exactos. 

Es por ello que se emplearán métodos inexactos, es decir que no pueden garantizar obtener la solución óptima, pero que que computacionalmente son aceptables.

Los algoritmos heurísticos se puede clasificar de la siguiente forma:
- **Progresivos**: Se comienza con el alineamiento de dos secuencias y de forma iterativa se van añadiendo el resto de secuencias al alineamiento. (ClustalW)
- **Iterativos**: Se realiza un alineamiento progresivo y se intenta mejorar el alineamiento moviendo, añadiendo o eliminando _gaps_. (Muscle)
- **Híbridos**: Combinan diferentes estrategias empleando información complementaria (información estructural de las proteinas o bases de datos con información de buenos alineamientos locales).

Para la utilización de los algoritmos heuristicos de alineamientos debemos instalar primero de forma local los algoritmos a utilizar, para nuestro caso , ClustalOmega y Muscle.

Una vez instalados, podremos realizar la ejecución de cada uno los programas de alineamiento mediante el módulo Align.Applications (https://biopython.org/docs/latest/api/Bio.Align.Applications.html#module-Bio.Align.Applications):

Applications.ClustalwCommandline

Applications.ClustalOmegaCommandline

Applications.MuscleCommandline

Sin embargo, desde el propio desarrollo de Biopython indican que se aconseja la ejecución de cada uno de los programas mediante subprocesos de Python (https://docs.python.org/3/library/subprocess.html). De esta forma, siempre trabajaremos con la ultima versión instalada y sin problemas de compatibilidad de las funciones de Biopython con el software instalado.

Veamos un par de ejemplos:

Comenzando por la ejecución del algoritmo de clustal para el primer exón de interés de BRCA1, el exón 5.

In [139]:
from Bio import AlignIO

#Ejecutar el algoritmo de Clustal
import subprocess

completed=subprocess.run(["clustalo", "-i",
                          "Datos/Secuencias filtradas/BRCA1 exon 5.fasta", "-o",
                          "Resultados/Resultados alineamiento cluscal/clustal algn BRCA1 exon5",
                          "-v","--force",
                          "--outfmt=clu"],capture_output=True)

print('returncode:\n\n\n', completed.stdout.decode('utf-8'))

returncode:


 Using 8 threads
Read 442 sequences (type: DNA) from Datos/Secuencias filtradas/BRCA1 exon 5.fasta
Using 77 seeds (chosen with constant stride from length sorted seqs) for mBed (from a total of 442 sequences)
Calculating pairwise ktuple-distances...
Ktuple-distance calculation progress done. CPU time: 0.57u 00:00:00.57 Elapsed: 00:00:01
mBed created 13 cluster/s (with a minimum of 1 and a soft maximum of 100 sequences each)
Distance calculation within sub-clusters done. CPU time: 0.49u 00:00:00.48 Elapsed: 00:00:00
Guide-tree computation (mBed) done.
Progressive alignment progress done. CPU time: 7.66u 00:00:07.65 Elapsed: 00:00:08
Alignment written to Resultados/Resultados alineamiento cluscal/clustal algn BRCA1 exon5



In [20]:

alineamiento_clustal=AlignIO.read("Resultados\Resultados alineamiento cluscal\clustal algn BRCA1 exon5","clustal")
print(alineamiento_clustal)

Alignment with 446 rows and 456 columns
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
---------ATTCTTTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932487.1
--------------TTCTACAAAAGGAAGTAAATTAAATTGTTC...--- MG932486.1
---------ATTCTTTCTACAAAAGGAAGT

De la misma forma aplicar el algoritmo iterativo de "Muscle":

In [140]:
import subprocess

completed=subprocess.run(["muscle.exe",
                          "-align", "Datos/Secuencias filtradas/BRCA1 exon 19.fasta",
                          "-output", "Resultados/Resultados_alineamiento_muscle/algn BRCA1 exon19.afa"],capture_output=True)

print('returncode:\n\n\n', completed.stdout.decode('utf-8'))


returncode:


 


In [26]:
alineamiento_muscle=AlignIO.read("Resultados/Resultados_alineamiento_muscle/algn BRCA1 exon19.afa","fasta")
print(alineamiento_muscle)

Alignment with 97 rows and 473 columns
GGGGA--G-A-CGTGTAAACC-CTAA-TG-T-C-TCA-G-G-T-...--A MH027271.1
GGGGG----G-CG-----GGC-CT-C-TG---A-TCT-G-C-G-...--A MH027296.1
CGGAA----C-T------GAA-CT-C-TC-TAA-CTG-C-A-A-...--A MH027293.1
AGAAA----A-G------TGA-CT-C-TC---T-TCT-C-A-G-...--A MH027249.1
CGTT-------C------GGA-CT-C-TC---T-TCT-G-C-G-...--A MH027213.1
CGGGG----C-C------GAACCC-C-TC-T-A-CCT-C-C-G-...--A MH027270.1
GGGGT--G-C-C------GAA-AT-C-TC-T-A-GCT-G-A-G-...--A MH027260.1
AGGAA----ACG------GCA-CT-C-TC---T-TCT-G-C-G-...--A MH027289.1
TTTT-----C-G------GAC-CC-C-TC---T-TCT-C-C-G-...--A MH027242.1
GGGTT----C-G------GGACCT-C-TG-T-A-TCT-C-C-G-...--G MH027269.1
AGGG---------------GA-CT-CCTC---T-TCT-C-C-G-...--A MH027254.1
GGGTA--C-C-C------TGA-CT-C-TC-A-C-TCT-G-C-G-...--A MH027300.1
GC-------G-G------GGA-CT-C-TC---T-TCT-A-C-G-...--G MH027214.1
GGTGT--C-C-T------TAA-CT-C-TC---T-TCT-C-C-G-...--A MH027287.1
AGGCC----C-C------CCT-CC-C-TC-----TCT-GGA-A-...--- MH027209.1
CGGGG--A-A-C------GGA-CT-C-TC-T

Para una mejor implementación y limpieza, relizamos la creación de un método genérico para la obtención de algoritmos con el nombre de "obtener_alineamiento"

In [150]:
def realizar_alineamiento(algoritmo,ruta_secuencias,ruta_alineamiento):
    if algoritmo == "clustalo":
        
        subprocess.run([algoritmo, "-i",ruta_secuencias, "-o",ruta_alineamiento,"-v","--force","--outfmt=clu"],capture_output=True)
        
    elif algoritmo== "muscle.exe":
        
        subprocess.run([algoritmo,"-align", ruta_secuencias,"-output", ruta_alineamiento],capture_output=True)
    
    #alineamientos=AlignIO.read(ruta_alineamiento,"fasta")
    
    #return alineamientos

In [152]:
def obtener_alineamiento(ruta_alineamiento):
    alineamientos=AlignIO.read(ruta_alineamiento,"fasta")
    return alineamientos

Primero creamos los archivos en nuestro equipo donde irán las secuencias.

Ahora realizamos los alineamientos con las secuencias filtradas para el algoritmo "Clustal":

In [158]:
realizar_alineamiento("clustalo","Datos/Secuencias filtradas/BRCA1 exon 5.fasta","Resultados/Resultados alineamiento clustal/clustal algn BRCA1 exon5.afa")
realizar_alineamiento("clustalo","Datos/Secuencias filtradas/BRCA1 exon 19.fasta","Resultados/Resultados alineamiento clustal/clustal algn BRCA1 exon19.afa")

realizar_alineamiento("clustalo","Datos/Secuencias filtradas/BRCA2 exon 2.fasta","Resultados/Resultados alineamiento clustal/clustal algn BRCA2 exon2.afa")
realizar_alineamiento("clustalo","Datos/Secuencias filtradas/BRCA2 exon 11.fasta","Resultados/Resultados alineamiento clustal/clustal algn BRCA2 exon11.afa")

realizar_alineamiento("clustalo","Datos/Secuencias filtradas/TP53 exon 5.fasta","Resultados/Resultados alineamiento clustal/clustal algn TP53 exon5.afa")
realizar_alineamiento("clustalo","Datos/Secuencias filtradas/TP53 exon 8.fasta","Resultados/Resultados alineamiento clustal/clustal algn TP53 exon8.afa")

realizar_alineamiento("clustalo","Datos/Secuencias filtradas/PIK3CA exon 9.fasta","Resultados/Resultados alineamiento clustal/clustal algn PIK3CA exon9.afa")
realizar_alineamiento("clustalo","Datos/Secuencias filtradas/PIK3CA exon 20.fasta","Resultados/Resultados alineamiento clustal/clustal algn PIK3CA exon20.afa")

Y guardamos cada resultado en una variable global.

In [165]:
c_algn_brca1_exon5=obtener_alineamiento("Resultados/Resultados alineamiento clustal/clustal algn BRCA1 exon5.afa")
c_algn_brca1_exon19=obtener_alineamiento("Resultados/Resultados alineamiento clustal/clustal algn BRCA1 exon19.afa")

c_algn_brca2_exon2=obtener_alineamiento("Resultados/Resultados alineamiento clustal/clustal algn BRCA2 exon2.afa")
c_algn_brca2_exon11=obtener_alineamiento("Resultados/Resultados alineamiento clustal/clustal algn BRCA2 exon11.afa")
                                        
c_algn_tp53_exon5=obtener_alineamiento("Resultados/Resultados alineamiento clustal/clustal algn TP53 exon5.afa")
c_algn_tp53_exon8=obtener_alineamiento("Resultados/Resultados alineamiento clustal//clustal algn TP53 exon8.afa")

c_algn_pik3ca_exon9=obtener_alineamiento("Resultados/Resultados alineamiento clustal/clustal algn PIK3CA exon9.afa")
c_algn_pik3ca_exon20=obtener_alineamiento("Resultados/Resultados alineamiento clustal/clustal algn PIK3CA exon20.afa")

ValueError: No records found in handle

Realizamos los correspondientes alineamientos con el método creado para el algoritmo de "Muscle", guardando cada uno de ellos en variables globales:

In [162]:
realizar_alineamiento("muscle.exe","Datos/Secuencias filtradas/BRCA1 exon 5.fasta","Resultados/Resultados alineamiento muscle/muscle algn BRCA1 exon5.afa")
realizar_alineamiento("muscle.exe","Datos/Secuencias filtradas/BRCA1 exon 19.fasta","Resultados/Resultados alineamiento muscle/muscle algn BRCA1 exon19.afa")

realizar_alineamiento("muscle.exe","Datos/Secuencias filtradas/BRCA2 exon 2.fasta","Resultados/Resultados alineamiento muscle/muscle algn BRCA2 exon2.afa")
realizar_alineamiento("muscle.exe","Datos/Secuencias filtradas/BRCA2 exon 11.fasta","Resultados/Resultados alineamiento muscle/muscle algn BRCA2 exon11.afa")

realizar_alineamiento("muscle.exe","Datos/Secuencias filtradas/PIK3CA exon 9.fasta","Resultados/Resultados alineamiento muscle/muscle algn PIK3CA exon9.afa")
realizar_alineamiento("muscle.exe","Datos/Secuencias filtradas/PIK3CA exon 20.fasta","Resultados/Resultados alineamiento muscle/muscle algn PIK3CA exon20.afa")

realizar_alineamiento("muscle.exe","Datos/Secuencias filtradas/TP53 exon 5.fasta","Resultados/Resultados alineamiento muscle/muscle algn TP53 exon5.afa")
realizar_alineamiento("muscle.exe","Datos/Secuencias filtradas/TP53 exon 8.fasta","Resultados/Resultados alineamiento muscle/muscle algn TP53 exon8.afa")

In [164]:
m_algn_brca1_exon5=obtener_alineamiento("Resultados/Resultados alineamiento muscle/muscle algn BRCA1 exon5.afa")
m_algn_brca1_exon19=obtener_alineamiento("Resultados/Resultados alineamiento muscle/muscle algn BRCA1 exon19.afa")

m_algn_brca2_exon2=obtener_alineamiento("Resultados/Resultados alineamiento muscle/muscle algn BRCA2 exon2.afa")
m_algn_brca2_exon11=obtener_alineamiento("Resultados/Resultados alineamiento muscle/muscle algn BRCA2 exon11.afa")

m_algn_tp53_exon5=obtener_alineamiento("Resultados/Resultados alineamiento muscle/muscle algn TP53 exon5.afa")
m_algn_tp53_exon8=obtener_alineamiento("Resultados/Resultados alineamiento muscle/muscle algn TP53 exon8.afa")

m_algn_pik3ca_exon9=obtener_alineamiento("Resultados/Resultados alineamiento muscle/muscle algn PIK3CA exon9.afa")
m_algn_pik3ca_exon20=obtener_alineamiento("Resultados/Resultados alineamiento muscle/muscle algn PIK3CA exon20.afa")